In [1]:
%matplotlib inline
project_dir = '../../'
data_dir = project_dir + 'data/'

import sys
sys.path.append(project_dir + 'NN/')

import numpy as np
import importlib
import time
import matplotlib.pyplot as plt
import MLP
import layer
importlib.reload(MLP)
importlib.reload(layer)
from MLP import MLP
from utils.preprocessing import split,Normalize
from utils.losses import MSE, MEE

In [2]:
raw_data=np.loadtxt("../../data/TR.csv",delimiter=",")[:,1:]
np.random.shuffle(raw_data)

input_data=raw_data[:,1:-2]
labels=raw_data[:,-2:]

In [15]:
import utils.grid_search
importlib.reload(utils.grid_search)
from utils.grid_search import grid_search

# Structure to test: 
n_feat = np.shape(labels)[1]
model1 = {'structure':[40, n_feat], 'func':['sigmoid', 'linear'], 'starting_points': [0.5]*2}
model2 = {'structure':[10, n_feat], 'func':['sigmoid','linear'], 'starting_points': [0.5]*2}
models = [model1, model2]
dict_models = {f'Model{i}': m for i, m in enumerate(models)} 

list_eta = [1e-1]
list_alpha = [1e-1, 1e-3, 0]
list_lamb = [1e-3, 0]
dict_params = {'eta':list_eta, 'alpha':list_alpha, 'lamb':list_lamb, 'epoch':[50], 'RMSProp' : [False]}

grid_results = grid_search(MLP, dict_models, dict_params, 
                           input_data, labels, MEE,
                           verbose = 1, kind = 'k_fold', k = 2)

0 on 12: Model Model0 --> Train parameters: {'eta': 0.1, 'alpha': 0.1, 'lamb': 0.001, 'epoch': 50, 'RMSProp': False}
Epoch 50:  [train MEE = 1.7589]  [val MEE = 1.8298]                              
Elapsed time: 0.5196859836578369 s
Epoch 50:  [train MEE = 1.9062]  [val MEE = 1.8920]                              
Elapsed time: 0.40140795707702637 s


1 on 12: Model Model0 --> Train parameters: {'eta': 0.1, 'alpha': 0.1, 'lamb': 0, 'epoch': 50, 'RMSProp': False}
Epoch 50:  [train MEE = 1.7637]  [val MEE = 1.8246]                              
Elapsed time: 0.537344217300415 s
Epoch 50:  [train MEE = 1.8404]  [val MEE = 1.8173]                              
Elapsed time: 0.6802167892456055 s


2 on 12: Model Model0 --> Train parameters: {'eta': 0.1, 'alpha': 0.001, 'lamb': 0.001, 'epoch': 50, 'RMSProp': False}
Epoch 50:  [train MEE = 1.8130]  [val MEE = 1.8679]                              
Elapsed time: 0.7506048679351807 s
Epoch 50:  [train MEE = 1.8785]  [val MEE = 1.8596]           

KeyboardInterrupt: 

In [4]:
grid_results

[{'train': {'eta': 0.1, 'alpha': 0.1, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [40, 2],
   'func': ['sigmoid', 'linear'],
   'starting_points': [0.5, 0.5]},
  'model_name': 'Pippo',
  'Error': 1.3224420861707522},
 {'train': {'eta': 0.1, 'alpha': 0.1, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [10, 2],
   'func': ['sigmoid', 'linear'],
   'starting_points': [0.5, 0.5]},
  'model_name': 'Pluto',
  'Error': 1.3321607953205634},
 {'train': {'eta': 0.1, 'alpha': 0, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [40, 2],
   'func': ['sigmoid', 'linear'],
   'starting_points': [0.5, 0.5]},
  'model_name': 'Pippo',
  'Error': 1.3480596983308235},
 {'train': {'eta': 0.1, 'alpha': 0, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [10, 2],
   'func': ['sigmoid', 'linear'],
   'starting_points': [0.5, 0.5]},
  'model_name': 'Pluto',
  'Error': 1.348380934177355},
 {'train': {'eta': 0.1, 'alpha': 0.001, 'lamb': 0, 'epoch': 500},
  'model': {'structure': [40, 2],
   'func'